In [3]:
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.animation as animation
from matplotlib.ticker import FormatStrFormatter
import numpy as np
import torch
import h5py
import astropy.units as u
#import astroalign as aa
#import sewpy
import re
from tqdm.notebook import tqdm
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import wcs

sys.path.append('../dancelib')
from residunet import ResidualEncoderUNet


# Download the data

In [14]:
data_url = 'https://deepdip.iap.fr/api/v1/item/66181d157a386bd3dc41d7f6/download'
data_dir = '../data/'
data_file = 'M92sim.1024x1024x160.fits'

# Download the data
print('Path: {}'.format(os.path.join(data_dir,data_file)))
if not os.path.exists(os.path.join(data_dir,data_file)):
    print('Downloading data...')
    import urllib.request
    urllib.request.urlretrieve(data_url,os.path.join(data_dir,data_file))


Path: ../data/M92sim.1024x1024x160.fits
